# Analyse Exploratoire du Dataset des Maisons
Dans ce notebook, nous allons analyser un dataset contenant des informations sur des maisons, explorer les relations entre les variables, et préparer les données pour la construction d'un modèle de régression.

## 1. Chargement des Librairies et du Dataset
Nous allons d'abord charger les librairies nécessaires et importer le dataset.

In [1]:
# lien de lancement sur Google Colab
# https://colab.research.google.com/github/Fred-Zang/AJC-ML_etat_art/blob/main/10-Regressions_v3.ipynb


# Importer les librairies nécessaires
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# lancement via Google Colab pour charger le dataset
url = "https://raw.githubusercontent.com/Fred-Zang/AJC-ML_etat_art/main/kc_house_data_modified.csv"
house_data = pd.read_csv(url)

# Charger le dataset en local
#house_data = pd.read_csv('data/kc_house_data_modified.csv')

# Afficher les premières lignes du dataset
house_data.head()


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180.0,5650,1.0,0,0.0,...,7,1180,0,1955,0,98178.0,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570.0,7242,2.0,0,0.0,...,7,2170,400,1951,1991,98125.0,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770.0,10000,1.0,0,0.0,...,6,770,0,1933,0,98028.0,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960.0,5000,1.0,0,0.0,...,7,1050,910,1965,0,98136.0,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680.0,8080,1.0,0,0.0,...,8,1680,0,1987,0,98074.0,47.6168,-122.045,1800,7503


# 2. Exploration Initiale du Dataset
Nous allons explorer le dataset pour comprendre sa structure, les types de variables qu'il contient, et obtenir un aperçu général des données.

In [ ]:
# Afficher des informations générales sur le dataset
house_data.info()

# Vérifier les statistiques descriptives des colonnes numériques
house_data.describe()


### Résumé des informations descriptives du dataset : (à compléter)
- id : identifiant
- price : Le prix des maisons
- bedrooms : Le nombre de chambres
- bathrooms : Le nombre de salles de bain
- sqft_living : La surface habitable
- floors : Le nombre d'étages
- waterfront : Indicateur de vue sur l'eau (0 : pas de vue, 1 : vue)
- yr_built : Les maisons datent de 1900 à 2015
- zipcode : Code postal des maisons
- lat et long : Coordonnées géographiques (latitude et longitude) des maisons.
- sqft_living15 : Surface habitable moyenne des 15 maisons les plus proches.
- sqft_lot15 : Surface moyenne du terrain des 15 maisons les plus proches.

## 3. Nettoyage, Préparation des Données et Normalisation
nous allons débugger et nettoyer le dataset afin de le rendre propre pour l'entraînement de modèles de régression.

### A. Gestion des Valeurs Manquantes
Nous allons d'abord identifier les lignes contenant des valeurs manquantes et les traiter.

In [ ]:
# Vérifier la présence de valeurs manquantes
missing_values = house_data.isnull().sum()
print(missing_values)


Nous avons plusieurs façons de gérer les valeurs manquantes :

1. Supprimer les lignes contenant des valeurs manquantes.
2. Remplacer les valeurs manquantes par la médiane ou la moyenne.


In [ ]:
# exemple pour remplacer par la médiane
#house_data['bedrooms'].fillna(house_data['bedrooms'].median(), inplace=True) # fillna() remplace les valeurs manquantes par la médiane de la colonne

house_data.drop(columns=['zipcode'], inplace=True)

# Supprimer les lignes où des NaN sont présents
house_data.dropna(inplace=True)

# Vérifier s'il reste des valeurs manquantes
print(house_data.isnull().sum())


### B. Suppression de la Colonne "id"
La colonne id est une simple clé unique qui n'apporte aucune information utile pour la modélisation. Nous allons donc la supprimer.

In [ ]:
# Suppression de la colonne 'id'
house_data.drop(columns=['id'], inplace=True)

# Vérifier que la colonne a bien été supprimée
print(house_data.columns)


### C.Reformater la Colonne "date" en Format DateTime
La colonne date est actuellement une chaîne de caractères. Nous allons la reformater en format datetime pour qu'elle soit plus facilement manipulable.

In [ ]:
# Reformater la colonne 'date' au format datetime
house_data['date'] = pd.to_datetime(house_data['date'], format='%Y%m%dT%H%M%S')

# Fusion des surfaces
house_data['total_sqft'] = house_data['sqft_living'] + house_data['sqft_above'] + house_data['sqft_basement'] + house_data['sqft_lot']

#Fusion des surfaces des voisins
house_data['neighbor_avg_sqft'] = house_data['sqft_living15'] + house_data['sqft_lot15']


# Afficher les premières lignes pour vérifier le changement
house_data.head()


### D. Détection et Gestion des Valeurs Aberrantes
Les valeurs aberrantes sont des points de données qui s'écartent anormalement des autres observations. Nous allons détecter ces valeurs dans les colonnes price et sqft_living, où nous avons introduit des valeurs extrêmes.

Étape 1 : Détection des valeurs aberrantes à l'aide d'un graphique
Nous allons utiliser des boîtes à moustaches (boxplots) pour détecter visuellement les valeurs aberrantes dans les colonnes price et sqft_living.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Sélectionner uniquement les colonnes numériques pour les boxplots
numeric_columns = house_data.select_dtypes(include=['float64', 'int64']).columns

# Créer une grille de sous-plots pour afficher les boxplots
plt.figure(figsize=(20, 12))
for i, column in enumerate(numeric_columns, 1):
    plt.subplot(4, 5, i)  # Créer une grille 4x5 (modifiable selon le nombre de colonnes)
    sns.boxplot(y=house_data[column])
    plt.title(f'Boxplot de {column}')

plt.tight_layout()  # Pour éviter les chevauchements
plt.show()



### E. Suppression des colonnes inutiles
- waterfront : La majorité des maisons n'ont pas de vue sur l'eau (la colonne est essentiellement binaire, avec très peu de maisons ayant une vue sur l'eau).
- view : Les boxplots montrent que cette colonne a des valeurs très concentrées.
- zipcode : inutile et nous gardons longitude et latitude

In [ ]:
# Suppression des colonnes 'waterfront', 'view', et 'zipcode'
house_data.drop(columns=['waterfront', 'view', 'yr_renovated', 'sqft_living', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15', 'sqft_lot'], inplace=True)

# Vérifier les colonnes restantes
print(house_data.columns)



### F. Nettoyage des Valeurs Aberrantes à l'aide de l'IQR
Ensuite, nous allons utiliser la méthode de l'IQR pour supprimer les valeurs aberrantes dans les colonnes restantes qui sont importantes pour le modèle. Nous nous concentrerons sur les colonnes avec de nombreuses valeurs aberrantes, comme :

- price
- sqft_living
- bathrooms
- bedrooms
- sqft_lot
- sqft_above
- sqft_basement


In [ ]:
# Liste des colonnes à nettoyer pour les valeurs aberrantes
columns_to_clean = ['price', 'bathrooms', 'bedrooms', 'total_sqft', 'neighbor_avg_sqft']

# Appliquer la méthode de l'IQR pour chaque colonne
for column in columns_to_clean:
    Q1 = house_data[column].quantile(0.25)
    Q3 = house_data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Filtrer les valeurs dans les bornes acceptables
    house_data = house_data[(house_data[column] >= lower_bound) & (house_data[column] <= upper_bound)]

# Vérifier les nouvelles dimensions du dataset après le nettoyage
print(house_data.shape)


## G. Normalisation
### Critères pour la normalisation :
- Régression linéaire : Ce modèle est sensible à l'échelle des données. Il est donc souvent recommandé de normaliser ou standardiser les colonnes si elles ont des plages de valeurs très différentes.
- Régression par forêt aléatoire : Ce modèle est moins sensible aux différences d'échelle, mais il peut tout de même être utile de normaliser certaines colonnes pour garantir une meilleure convergence et des performances plus constantes.

In [ ]:
# refaire un describe() final après notre nettoyage pour décider les features à normaliser ou non
house_data.describe()

### Colonnes à normaliser :
- total_sqft
  

- neighbor_avg_sqft

  
### Colonnes probablement sans besoin de normalisation :
- bedrooms et bathrooms : Ce sont des compteurs discrets avec une plage limitée.
  
- floors : Les valeurs sont assez faibles (entre 1 et 3.5).
  
- condition, grade : Ce sont des colonnes catégoriques ordinales, qui ne nécessitent pas de normalisation.
  
- yr_built : Les années peuvent être traitées telles quelles, mais la standardisation peut être envisagée si leur impact est important dans le modèle
  
- lat et long : Ce sont des coordonnées géographiques, nous voulons les concerver intactes

### StandardScaler
- Cette méthode transformera les colonnes en ayant une moyenne de 0 et un écart-type de 1

In [ ]:
from sklearn.preprocessing import StandardScaler

# Liste des colonnes à normaliser
columns_to_normalize = ['total_sqft', 'neighbor_avg_sqft']

# Initialiser le StandardScaler
scaler = StandardScaler()

# Appliquer la normalisation uniquement sur les colonnes sélectionnées
house_data[columns_to_normalize] = scaler.fit_transform(house_data[columns_to_normalize])

# Vérifier les résultats
print(house_data[columns_to_normalize].head())


In [ ]:
house_data.head()

# 4. Analyse des Corrélations entre les Variables
Pour mieux comprendre la relation entre les variables, nous allons analyser la matrice de corrélation et voir quelles variables sont les plus corrélées avec le prix des maisons.

In [ ]:
df = house_data.drop(['lat', 'long'], axis=1)

In [ ]:
# Calculer la matrice de corrélation
correlation_matrix = df.corr()

# Afficher la corrélation des variables avec le prix
print(correlation_matrix['price'].sort_values(ascending=False))

# Visualisation de la matrice de corrélation
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Matrice de Corrélation')
plt.show()


## 6. modèle de régression Gradient Boosting
![image.png](attachment:580f0a4b-022d-4970-b155-d3f042daa681.png)
- Un modèle de Gradient Boosting avec 100 arbres de décision (n_estimators=100) et une profondeur maximale de 3 (max_depth=3) a été utilisé. Ces paramètres peuvent être ajustés pour améliorer les performances.
- A. Diviser les données en ensembles d'entraînement et de test.

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
house_data.describe()


In [ ]:
house_data_selected = house_data.drop(columns=['date','lat', 'long'])

In [ ]:
# Séparer les features (X) et la cible (y)
X = house_data_selected.drop(columns=['price'])
y = house_data_selected['price']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer et entraîner le modèle de Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(X_train, y_train)

# Évaluer la performance du modèle
y_pred_gb = gb_model.predict(X_test)

# Calculer le coefficient de détermination R² et le RMSE
r2_gb = r2_score(y_test, y_pred_gb)
rmse_gb = np.sqrt(mean_squared_error(y_test, y_pred_gb))

# Afficher les résultats
print(f"Coefficient de détermination R² (Gradient Boosting) : {r2_gb:.4f}")
print(f"Root Mean Squared Error (RMSE - Gradient Boosting) : {rmse_gb:.2f} USD")






In [ ]:

# Faire des prédictions sur de nouveaux exemples
new_samples = pd.DataFrame({
    'bedrooms' : [5,6,8],
    'bathrooms': [2, 3, 4],
    'floors' : [1, 2, 2],
    'condition' : [2, 3, 4],
    'grade': [7, 9, 10],
    'yr_built': [1990, 2005, 2015],
    'total_sqft': [11800, 12500, 13500],
    'neighbor_avg_sqft': [5000, 8000, 12000]

})

# Normaliser les nouvelles données
new_samples[columns_to_normalize[:6]] = scaler.transform(new_samples[columns_to_normalize[:6]])

# Faire la prédiction avec le modèle Gradient Boosting
new_predictions = gb_model.predict(new_samples)

# Afficher les résultats
for i, pred in enumerate(new_predictions):
    print(f"Exemple {i+1} :")
    print(new_samples.iloc[i])
    print(f"Prix prédit : {pred:.2f} USD\n")

## Interprétation des résultats :
1. Coefficient de détermination R² : 0.5734 :
- Le R² = 0.5734 signifie que le modèle de Gradient Boosting explique environ 57,34 % de la variance dans les prix des maisons.
- C'est une amélioration significative par rapport au R² précédent de 0.1768, ce qui indique que les regroupements de features et la réduction de la colinéarité ont aidé le modèle à mieux capturer les relations dans les données.
- Bien que 57,34 % ne soit pas parfait, c'est un bon résultat, surtout pour un problème complexe comme la prédiction des prix des maisons, où de nombreuses variables externes (comme l'emplacement exact, la qualité des matériaux, ou les tendances du marché) ne sont pas forcément incluses dans les données.
  
2. Root Mean Squared Error (RMSE) : 133 346.41 USD :
- Le RMSE représente l'erreur quadratique moyenne entre les prix réels et les prix prédits, mesurée dans les mêmes unités que la variable cible (USD).
- Un RMSE de 133 346.41 USD signifie que le modèle fait en moyenne une erreur de 133 346 USD lors de la prédiction des prix des maisons. C'est une amélioration substantielle par rapport à l'erreur précédente (556 744.61 USD).
- Si l'on compare cette valeur à la distribution des prix des maisons (qui varient probablement entre quelques centaines de milliers et plusieurs millions), cet RMSE est raisonnable pour ce type de modèle.
  
3. Interprétation globale :
- Amélioration : Les regroupements des features liés aux surfaces ont permis de réduire la colinéarité et de simplifier la structure des données, ce qui a conduit à une meilleure performance du modèle.
- R² : Avec un R² de 0.5734, le modèle capture une part importante de la variance des prix, mais il reste des aspects du problème que le modèle n'explique pas encore bien. Cela peut être dû à des facteurs externes (comme des informations manquantes sur la qualité de la maison, des données environnementales, etc.).
- RMSE : L'erreur moyenne reste importante, mais elle est significativement plus faible, ce qui montre que les ajustements ont permis de réduire l'erreur de prédiction.
  
4. Suggestions pour continuer l'amélioration :
- Affiner les hyperparamètres : Maintenant que le modèle fonctionne mieux, tu pourrais encore affiner les hyperparamètres du modèle de Gradient Boosting (par exemple, augmenter le nombre d'estimations n_estimators, ajuster la profondeur des arbres max_depth, ou réduire légèrement le taux d'apprentissage learning_rate).
- Ajouter de nouvelles features : Si des données supplémentaires sont disponibles, comme des informations plus précises sur l'emplacement ou les matériaux utilisés dans la construction des maisons, cela pourrait aider à améliorer encore le modèle.

In [ ]:
import folium

# 7. Créer une carte intéractive de localisation

**conda install folium** ou bien **pip install folium**

Nous allons créer une carte centrée sur la moyenne des coordonnées latitude et longitude, puis y placer des points représentant chaque maison.

- folium.Map() : Crée une carte centrée sur la moyenne des coordonnées lat et long avec un niveau de zoom initial (ici zoom_start=10).
- MarkerCluster() : Ce plugin permet de regrouper les marqueurs lorsqu'ils se trouvent trop près les uns des autres. Cela améliore la lisibilité de la carte.
- folium.Marker() : Crée des marqueurs pour chaque maison, en utilisant les coordonnées latitude et longitude. Le popup affiche des informations comme le prix et la surface de la maison.
- Utilisation de FastMarkerCluster : Folium propose déjà une solution pour gérer un grand nombre de points de manière plus efficace. Le FastMarkerCluster est une alternative au MarkerCluster standard et peut accélérer considérablement la création des marqueurs, car il utilise des techniques de regroupement plus rapides.

In [ ]:
import folium
from folium.plugins import FastMarkerCluster

# Calculer la position centrale de la carte (moyenne des latitudes et longitudes)
map_center = [house_data['lat'].mean(), house_data['long'].mean()]

# Créer la carte centrée sur la région des maisons
house_map = folium.Map(location=map_center, zoom_start=10)

# Créer une liste de tuples (latitude, longitude) pour FastMarkerCluster
locations = list(zip(house_data['lat'], house_data['long']))

# Utiliser FastMarkerCluster pour ajouter les points
FastMarkerCluster(locations).add_to(house_map)

# Sauvegarder la carte ou l'afficher
house_map.save('data/house_map_fast.html')
house_map


## A. Colorer les Points en Fonction du Prix des Maisons
Nous pouvons également ajouter une fonctionnalité qui colorie les points en fonction du prix des maisons pour une meilleure visualisation de la répartition des prix dans la région.
- price_color() : Cette fonction attribue une couleur différente à chaque point en fonction du prix des maisons (vert pour les prix bas, orange pour les prix moyens, rouge pour les prix élevés).
- folium.CircleMarker() : Utilisé à la place de Marker pour créer des marqueurs de forme circulaire avec des couleurs personnalisées.

In [ ]:
import folium

# Calculer la position centrale de la carte (moyenne des latitudes et longitudes)
map_center = [house_data['lat'].mean(), house_data['long'].mean()]

# Créer la carte centrée sur la région des maisons
house_map = folium.Map(location=map_center, zoom_start=10)

# Créer une fonction pour attribuer des couleurs en fonction du prix
def price_color(price):
    if price < 300000:
        return 'green'
    elif 300000 <= price < 600000:
        return 'orange'
    else:
        return 'red'

# Limiter à un sous-ensemble de points pour accélérer la carte (par exemple, afficher seulement 1000 points)
sampled_data = house_data #.sample(1000)  # Vous pouvez ajuster ce nombre

# penser à rajouter
# Utiliser FastMarkerCluster pour ajouter les points
# FastMarkerCluster(locations).add_to(house_map)

# Ajouter des marqueurs colorés en fonction du prix
for idx, row in sampled_data.iterrows():  # ou sur house_data
    folium.CircleMarker(
        location=[row['lat'], row['long']],
        radius=5,
        color=price_color(row['price']),
        fill=True,
        fill_opacity=0.7,
        popup=f"Prix: {row['price']} USD\nSurface: {row['total_sqft']} sqft"
    ).add_to(house_map)

# Sauvegarder la carte et l'afficher
house_map.save('data/house_map_colored.html')
house_map
